In [ ]:
import pandas as pd
import os
import numpy as np
import pandas as pda
import missingno as msno
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets 
from torchvision import transforms
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [4]:
df_games = pd.read_csv('df_games_output.csv')

In [5]:
df_games

,gameId,gameDate,hometeamName,awayteamName,homeScore,awayScore,winner,gameType,attendance,win_home,...,threePointersPercentage_away,freeThrowsAttempted_away,freeThrowsMade_away,freeThrowsPercentage_away,reboundsDefensive_away,reboundsOffensive_away,reboundsTotal_away,foulsPersonal_away,turnovers_away,plusMinusPoints_away
0,22400627,2025-03-19 21:00:00,Jazz,Wizards,0,0,Wizards,Regular Season,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22400828,2025-02-24 22:00:00,Kings,Hornets,130,88,Kings,Regular Season,17832.0,1,...,0.250,24.0,18.0,0.750,22.0,15.0,37.0,16.0,19.0,-42.0
2,22400827,2025-02-24 21:00:00,Jazz,Trail Blazers,112,114,Trail Blazers,Regular Season,18175.0,0,...,0.318,24.0,18.0,0.750,35.0,16.0,51.0,16.0,13.0,2.0
3,22400826,2025-02-24 20:00:00,Thunder,Timberwolves,128,131,Timberwolves,Regular Season,18203.0,0,...,0.281,39.0,34.0,0.872,43.0,11.0,54.0,23.0,13.0,3.0
4,22400825,2025-02-24 19:30:00,Hawks,Heat,98,86,Hawks,Regular Season,16189.0,1,...,0.175,27.0,25.0,0.926,31.0,14.0,45.0,9.0,17.0,-12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50576,48400131,1985-04-18 19:00:00,Pistons,Nets,125,105,Pistons,Playoffs,NaN,1,...,0.000,28.0,19.0,0.679,30.0,9.0,39.0,28.0,15.0,-20.0
50577,48400141,1985-04-18 19:00:00,Lakers,Suns,142,114,Lakers,Playoffs,NaN,1,...,0.200,16.0,11.0,0.688,31.0,11.0,42.0,22.0,20.0,-28.0
50578,48400151,1985-04-18 19:00:00,Nuggets,Spurs,141,111,Nuggets,Playoffs,NaN,1,...,0.000,41.0,33.0,0.805,23.0,10.0,33.0,33.0,18.0,-30.0
50579,48400171,1985-04-18 19:00:00,Mavericks,Trail Blazers,139,131,Mavericks,Playoffs,NaN,1,...,0.375,17.0,14.0,0.824,41.0,12.0,53.0,34.0,22.0,-8.0


In [41]:
df = df_games

y = df['winner']

df = df.drop(columns=['gameDate'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


ValueError: could not convert string to float: 'Kings'

In [24]:
print(X.shape)  # when fitting StandardScaler


(96496, 37)


In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load your dataset
df = df_games

# Drop rows with missing values in important columns
df.dropna(subset=['homeScore', 'awayScore'], inplace=True)

# Define statistical columns to use for training
stat_cols = [
    'assists', 'blocks', 'steals',
    'fieldGoalsAttempted', 'fieldGoalsMade', 'fieldGoalsPercentage',
    'threePointersAttempted', 'threePointersMade', 'threePointersPercentage',
    'freeThrowsAttempted', 'freeThrowsMade', 'freeThrowsPercentage',
    'reboundsDefensive', 'reboundsOffensive', 'reboundsTotal',
    'foulsPersonal', 'turnovers', 'plusMinusPoints'
]

# Create feature column names for home and away teams
home_cols = [f"{stat}_home" for stat in stat_cols]
away_cols = [f"{stat}_away" for stat in stat_cols]
feature_cols = home_cols + away_cols

# Drop rows with missing feature data
df.dropna(subset=feature_cols, inplace=True)

# Define X and y
X = df[feature_cols]
y = (df['homeScore'] > df['awayScore']).astype(int)  # 1 if home wins, 0 if away wins

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build the neural network
model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20


/Users/parthshukla/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


988/988 ━━━━━━━━━━━━━━━━━━━━ 1s 474us/step - accuracy: 0.9406 - loss: 0.1530 - val_accuracy: 0.9970 - val_loss: 0.0133
Epoch 2/20
988/988 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - accuracy: 0.9978 - loss: 0.0091 - val_accuracy: 0.9990 - val_loss: 0.0060
Epoch 3/20
988/988 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - accuracy: 0.9996 - loss: 0.0028 - val_accuracy: 0.9994 - val_loss: 0.0042
Epoch 4/20
988/988 ━━━━━━━━━━━━━━━━━━━━ 0s 476us/step - accuracy: 0.9998 - loss: 0.0012 - val_accuracy: 0.9995 - val_loss: 0.0034
Epoch 5/20
988/988 ━━━━━━━━━━━━━━━━━━━━ 0s 408us/step - accuracy: 0.9997 - loss: 9.6983e-04 - val_accuracy: 0.9990 - val_loss: 0.0045
Epoch 6/20
988/988 ━━━━━━━━━━━━━━━━━━━━ 0s 454us/step - accuracy: 0.9991 - loss: 0.0024 - val_accuracy: 0.9999 - val_loss: 0.0027
Epoch 7/20
988/988 ━━━━━━━━━━━━━━━━━━━━ 0s 433us/step - accuracy: 1.0000 - loss: 1.1188e-04 - val_accuracy: 0.9999 - val_loss: 0.0026
Epoch 8/20
988/988 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - accuracy: 1.0000 - loss: 5.4342e-05 -

In [31]:
# ✅ Prediction Function
def predict_winner_neutral(teamA, teamB):
    # Get data for each team as home or away
    A_home = df[df['hometeamName'] == teamA][home_cols]
    A_away = df[df['awayteamName'] == teamA][away_cols]
    B_home = df[df['hometeamName'] == teamB][home_cols]
    B_away = df[df['awayteamName'] == teamB][away_cols]

    if A_home.empty and A_away.empty:
        print(f"No data for team: {teamA}")
        return
    if B_home.empty and B_away.empty:
        print(f"No data for team: {teamB}")
        return

    # Rename away columns to match home columns
    A_away.columns = home_cols
    B_away.columns = home_cols

    # Combine and average stats
    A_stats = pd.concat([A_home, A_away]).mean()[home_cols]
    B_stats = pd.concat([B_home, B_away]).mean()[home_cols]

    # Combine team A + B stats (A treated as home, B as away for prediction)
    combined = pd.concat([A_stats, B_stats]).values.reshape(1, -1)

    # Match scaler input shape
    combined_scaled = scaler.transform(combined)

    # Predict
    prob = model.predict(combined_scaled)[0][0]
    print(f"{teamA} vs. {teamB}")
    print(f"{teamA} win probability: {prob:.2%}")
    print(f"🏀 Predicted Winner: {'🟢 ' + teamA if prob >= 0.5 else '🔴 ' + teamB}")

# 🧪 Example usage:
predict_winner_neutral("Warriors", "Rockets")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Warriors vs. Rockets
Warriors win probability: 0.00%
🏀 Predicted Winner: 🔴 Rockets


/Users/parthshukla/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
